In [ ]:
# Colab setup code block
try:
    import google.colab
    print("Running on Google Colab. Setting up environment...")
    
    import os
    if not os.path.exists('CMAI-E91-Students'):
        !git clone https://github.com/algolab-quantique/CMAI-E91-Students.git

    os.chdir('CMAI-E91-Students/workshops')
    !pip install -r ../requirements.txt -q
    print("Environment setup complete!")
except ImportError:
    print("Not running on Google Colab. Skipping setup.")


# CMAI E91 Quantum Key Distribution Workshop

**Duration:** 3 hours (guided + hands-on + assignment)  
**Level:** Beginner  

This unified workshop notebook covers:
1. **Part 1:** Understanding entanglement and the CHSH Bell Inequality
2. **Part 2:** Implementing the complete E91 QKD Protocol
3. **Part 3:** Coding Challenge (Assignment)

---

# CHSH Bell Inequality - Hands-On Workshop

**Learning Objectives:**
- Create entangled Bell states  
- Implement measurement-basis transformations  
- Calculate correlations and the CHSH value  
- Verify that quantum entanglement violates classical limits  

**Goal:** Demonstrate that entangled pairs can produce $|S| > 2$.


**Roadmap (functions you’ll implement):**
`create_bell_pair_singlet_state()` → `apply_basis_transformation()` → `measure_bell_pair()` → `organize_measurements_by_basis()` → `calculate_correlations()` → `calculate_chsh_value()`

---

## CHSH Bell Inequality: Key Points (Quick Reference)

The CHSH test checks whether two systems (e.g., qubits) are **quantum entangled**.

### What the CHSH value means

| System | CHSH Value $\|S\|$ |
|--------|-------------|
| Classical (hidden variables) | $\|S\| \le 2$ |
| **Quantum (entangled)** | **$2 < \|S\| \le 2\sqrt{2} \approx 2.83$** |

If $|S| > 2$ → **Quantum entanglement confirmed!**

In practice, a statistically significant violation (beyond the error bars) is required to experimentally conclude that the state is entangled.


### Which measurement bases are used?

The test uses **four combinations** of measurement bases for Alice and Bob. Each side uses **two bases**:
- **Alice:** $a_1 = 0^\circ$, $a_2 = 90^\circ$
- **Bob:** $b_1 = 45^\circ$, $b_2 = 135^\circ$

Pairs used in CHSH: $(a_1,b_1)$, $(a_1,b_2)$, $(a_2,b_1)$, $(a_2,b_2)$

Bases are applied by `apply_basis_transformation()` and measured in `measure_bell_pair()`.

### Correlation for each basis pair
Computed by `calculate_correlations()`
$$
E(a,b) = \frac{N_{00} + N_{11} - N_{01} - N_{10}}{N_{00} + N_{11} + N_{01} + N_{10}}
$$

Where $N_{xy}$ is the **count** of measurement outcomes $x$ for Alice and $y$ for Bob (e.g., $N_{00}$ is how many times the result was $00$).

Counts grouped by `organize_measurements_by_basis()`.

### CHSH formula

$$
S = E(a_1,b_1) - E(a_1,b_2) + E(a_2,b_1) + E(a_2,b_2)
$$

Computed by `calculate_chsh_value()`.

---

## Bridge between the presentation and the notebook: physics → simulation

**Real experiment:** entangled photons + polarizers/waveplates that set the measurement angles.  
**In practice:** we rotate optical elements to choose the measurement basis.  
**In this notebook:** Bell pairs + rotation gates to simulate those angles in Qiskit.

So when you see **0°, 45°, 90°, 135°** below, think **polarizer angles** in the lab.

---


## Setup

Run this cell to import all required libraries and helper functions.

In [1]:
# Standard imports
import random
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Set seed for reproducibility
GLOBAL_SEED = 91
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

# Initialize simulator
aer_simulator = AerSimulator()
MANUAL_SIMULATOR_SEED_COUNTER = GLOBAL_SEED

# Constants
ALICE_BELL_BASES = ['0', '90']   # Alice's measurement bases for CHSH
BOB_BELL_BASES = ['45', '135']   # Bob's measurement bases for CHSH
BELL_INEQUALITY_THRESHOLD = 2.0  # Classical limit

print("Setup complete!")
print(f"Alice's bases: {ALICE_BELL_BASES}")
print(f"Bob's bases: {BOB_BELL_BASES}")

Setup complete!
Alice's bases: ['0', '90']
Bob's bases: ['45', '135']


In [2]:
# Helper function to run circuits (provided - just run this cell)
def run_circuit(circ: QuantumCircuit, shots=1) -> dict:
    """
    Run a quantum circuit on the AerSimulator and return the counts.
    """
    global MANUAL_SIMULATOR_SEED_COUNTER
    global aer_simulator
    
    current_run_seed = MANUAL_SIMULATOR_SEED_COUNTER
    MANUAL_SIMULATOR_SEED_COUNTER += 1
    
    circ = transpile(circ, aer_simulator)
    result = aer_simulator.run(circ, shots=shots, seed_simulator=current_run_seed).result()
    return result.get_counts(circ)

print("Helper function loaded!")

Helper function loaded!


---
## 1) Create a Bell State

We start by preparing an **entangled pair**. The singlet state guarantees perfect anti‑correlation, which is what gives a CHSH violation.

The **singlet state** $|\Psi^-\rangle = (|01\rangle - |10\rangle)/\sqrt{2}$ is used in the original E91 protocol.

**Recipe:**
1. Apply Hadamard (H) to qubit 0
2. Apply CNOT with control=0, target=1
3. Apply X then Z to qubit 1 (to get $|\Psi^-\rangle$ specifically)

Exercise: Implement `create_bell_pair_singlet_state()`

In [ ]:
def create_bell_pair_singlet_state() -> QuantumCircuit:
    """
    Create the Bell singlet state |Ψ-⟩ = (|01⟩ - |10⟩)/√2
    
    Returns:
        QuantumCircuit with 2 qubits in the singlet Bell state
    """
    # TODO: Create a quantum circuit with 2 qubits
    qc = None  # Replace with: QuantumCircuit(2)
    
    # Step 1: Create superposition
    # TODO: Apply Hadamard to qubit 0
    
    # Step 2: Entangle qubits
    # TODO: Apply CNOT (control=0, target=1)
    
    # Step 3: Transform to |Ψ-⟩ state
    # TODO: Apply X gate to qubit 1
    # TODO: Apply Z gate to qubit 1
    
    return qc

In [ ]:
# Test your Bell state
bell_state = create_bell_pair_singlet_state()
bell_state.draw('mpl')

In [ ]:
# Verify by measuring many times
test_qc = bell_state.copy()
test_qc.measure_all()
counts = run_circuit(test_qc, shots=1000)
print(f"\nMeasurement results: {counts}")
print("\nExpected: roughly 50% |01⟩ and 50% |10⟩ (never |00⟩ or |11⟩)")


---
## 2) Measurement Basis Transformation

We always measure in the Z‑basis, so to simulate other bases we **rotate first**, then measure.

| Basis | Angle | Gate |
|-------|-------|------|
| Z-basis | 0° | None |
| 45° | 45° | Ry(-π/4) |
| X-basis | 90° | H |
| 135° | 135° | Ry(-3π/4) |

Exercise: Implement `apply_basis_transformation()`

### Quick intuition (bases)

- **Same basis** → strong (anti-)correlation for entangled pairs.
- **Different bases** → results look more random.

This is exactly what the polarizer angles do in the photon experiment.

---

In [ ]:
def apply_basis_transformation(circuit: QuantumCircuit, qubit_index: int, basis: str) -> QuantumCircuit:
    """
    Apply a measurement basis transformation to a qubit.
    
    Args:
        circuit: The quantum circuit to transform
        qubit_index: Index of the qubit (0 or 1)
        basis: '0' (Z), '45', '90' (X), or '135'
    
    Returns:
        Copy of the circuit with basis transformation applied
    """
    # Create a copy of the circuit
    transformed = circuit.copy()
    
    if basis == '0':    # Z-basis (0°)
        pass  # No transformation needed
    
    elif basis == '90':  # X-basis (90°)
        # TODO: Apply Hadamard gate on the specified qubit (qubit_index)
        pass
    
    elif basis == '45':  # 45° basis
        # TODO: Apply Ry(-π/4) rotation
        # Hint: transformed.ry(-np.pi/4, qubit_index)
        pass
    
    elif basis == '135':  # 135° basis
        # TODO: Apply Ry(-3π/4) rotation
        pass
    
    else:
        raise ValueError(f"Unknown basis: {basis}")
    
    return transformed

In [ ]:
#  Test basis transformations
test_qc = QuantumCircuit(1)

print("Testing basis transformations:")
for basis in ['0', '45', '90', '135']:
    result = apply_basis_transformation(test_qc, 0, basis)
    print(f"\nBasis {basis}°:")
    print(result.draw())

---
## 3) Measure Bell Pairs

Each entangled pair can be measured **only once** because measurement collapses the state.

Now we combine Bell state creation with basis transformations to measure entangled pairs.

Exercise: Implement `measure_bell_pair()`

In [ ]:
def measure_bell_pair(circuit: QuantumCircuit, alice_basis: str, bob_basis: str) -> str:
    """
    Measure a Bell pair with Alice and Bob using specified bases.
    
    Args:
        circuit: Bell pair circuit
        alice_basis: Alice's measurement basis ('0', '45', '90')
        bob_basis: Bob's measurement basis ('45', '90', '135')
    
    Returns:
        str: Measurement result ('00', '01', '10', or '11')
    """
    # Create a copy of the circuit
    meas_qc = circuit.copy()
    
    # TODO: Apply Alice's basis transformation to qubit 0
    
    # TODO: Apply Bob's basis transformation to qubit 1
    
    # Add measurements to all qubits
    meas_qc.measure_all()
    
    # Run the circuit and return the result
    counts = run_circuit(meas_qc, shots=1)
    
    return list(counts.keys())[0]

In [ ]:
# Test Bell pair measurement
bell_state = create_bell_pair_singlet_state()
from collections import Counter


# Test multiple measurements
print("Testing Bell pair measurements (1000 runs each)")

print("\nDifferent bases (Alice 0°, Bob 45°) - Random results:")
results_diff = [measure_bell_pair(bell_state, '0', '45') for _ in range(200)]
print(f"  Counts: {dict(Counter(results_diff))}")

print("\nSame bases (Alice 0°, Bob 0°) - Perfect Anti-correlation:")
# For Singlet State |Ψ-⟩, same-basis measurements must be opposite ('01' or '10')
results_same = [measure_bell_pair(bell_state, '0', '0') for _ in range(200)]
print(f"  Counts: {dict(Counter(results_same))}")


---
## 4) Run Bell Test Measurements

To estimate CHSH, we need many trials with **randomly chosen bases** to sample all four basis pairs.

For the CHSH test, we need to:
1. Create many Bell pairs
2. Randomly choose bases for Alice and Bob
3. Record all measurements

Exercise: Implement `run_bell_test_measurements()`

In [ ]:
def run_bell_test_measurements(list_bell_pairs, list_alice_bases=ALICE_BELL_BASES, list_bob_bases=BOB_BELL_BASES):
    """
    Run measurements on Bell pairs using random basis choices.
    
    Args:
        list_bell_pairs: List of Bell pair circuits
        list_alice_bases: Alice's possible bases
        list_bob_bases: Bob's possible bases
    
    Returns:
        Tuple of (results, alice_bases_used, bob_bases_used)
    """
    results = []
    alice_bases_used = []
    bob_bases_used = []
    
    for qc in list_bell_pairs:
        # TODO: Randomly choose Alice's basis from list_alice_bases
        a_base = None  # Hint: random.choice(list_alice_bases)
        
        # TODO: Randomly choose Bob's basis from list_bob_bases
        b_base = None # Hint: random.choice(list_bob_bases)
        
        # TODO: Measure the Bell pair qc with a_base and b_base
        result = None  # Hint: measure_bell_pair(qc, a_base, b_base)
        
        # Record everything
        results.append(result)
        alice_bases_used.append(a_base)
        bob_bases_used.append(b_base)
    
    return results, alice_bases_used, bob_bases_used

---
## 5) Organize Results by Basis Pair

We must **group results by basis pair** so we can compute $E(a,b)$ for each combination.



In [ ]:
def organize_measurements_by_basis(results, alice_bases, bob_bases):
    """
    Organize measurements by basis pairs.
    
    Returns:
        Dict mapping (alice_basis, bob_basis) to {'00': count, '01': count, ...}
    """
    # Get unique bases
    unique_alice = list(set(alice_bases))
    unique_bob = list(set(bob_bases))
    
    # Initialize counts dictionary
    counts = {}
    for a in unique_alice:
        for b in unique_bob:
            counts[(a, b)] = {'00': 0, '01': 0, '10': 0, '11': 0}
    
    # Loop through results and increment the counter for the used basis pair
    for i, result in enumerate(results):
        a_base = alice_bases[i]
        b_base = bob_bases[i]
        
        # Check if result is valid (just in case), then increment
        if result in counts[(a_base, b_base)]:
            counts[(a_base, b_base)][result] += 1
            
    return counts


---
## 6) Calculate Correlations

Correlation tells us whether Alice and Bob are **usually the same or different**. It always lies between **−1 and +1**.

**Counts formula (what we use in code):**
$$
E(a,b)=\frac{N_{00}+N_{11}-N_{01}-N_{10}}{N_{00}+N_{11}+N_{01}+N_{10}}
$$

### Link to the slide definition (±1 outcomes)

In the slides, each result is converted to **±1** and then averaged:
- Same bits ($00$ or $11$) → $+1$  
- Different bits ($01$ or $10$) → $-1$

So the correlation is just the **average of the product**:
$$
E(a,b)=\langle A\cdot B\rangle=\frac{1}{N}\sum_{k=1}^{N} A_k B_k
$$

The counts formula above is **exactly the same thing**, just written using totals of each outcome.

---

In [ ]:
def calculate_correlations(measurements):
    """
    Calculate E(a,b) for each basis pair.
    
    Args:
        measurements: Dict from organize_measurements_by_basis()
    
    Returns:
        Dict mapping (alice_basis, bob_basis) to correlation value
    """
    correlations = {}
    
    for basis_pair, results in measurements.items():
        total = sum(results.values()) # Total measurements: results['00'] + results['01'] + results['10'] + results['11']
        
        if total > 0: # to avoid division by zero :p
              
            # Formula: (Same - Different) / Total
            E = 0 # replace with formula ...
            correlations[basis_pair] = E
        else:
            correlations[basis_pair] = 0
    
    # Print for debugging
    print("Correlations:")
    for basis, corr in correlations.items():
        print(f"  E{basis} = {corr:.4f}")
    
    return correlations


---
## 7) Calculate CHSH Value

CHSH combines the four correlations in a **specific pattern** that exposes quantum behavior.

**Formula:** $S = E(a_1,b_1) - E(a_1,b_2) + E(a_2,b_1) + E(a_2,b_2)$

With our bases:
- $a_1 = 0^\circ$, $a_2 = 90^\circ$ (Alice)
- $b_1 = 45^\circ$, $b_2 = 135^\circ$ (Bob)

Exercise: Implement `calculate_chsh_value()`

In [3]:
def calculate_chsh_value(correlations, alice_bases=ALICE_BELL_BASES, bob_bases=BOB_BELL_BASES):
    """
    Calculate the CHSH Bell parameter S.

    Args:
        correlations: Dictionary mapping (alice_basis, bob_basis) to correlation value E(a, b).
        example:
        correlations[(a1, b1)] = E(a1, b1)
        alice_bases: List of Alice's two measurement bases (default: ['0', '90']).
        bob_bases: List of Bob's two measurement bases (default: ['45', '135']).

    
    Returns:
        float: |S| value
    """
    a1, a2 = alice_bases  # '0', '90'
    b1, b2 = bob_bases    # '45', '135'

    # Note: correlations[(a1, b1)] gives E(a1, b1)
    
    # TODO: Calculate S using the CHSH formula
    # S = E(a1,b1) - E(a1,b2) + E(a2,b1) + E(a2,b2)
    # HINT: Notice where the minus sign is! This formula is specific to the |Ψ-⟩ state.
    S = 0  # Replace with actual calculation
    
    return abs(S)


def check_bell_inequality(chsh_value):
    """Returns True if Bell inequality is violated (quantum entanglement detected)."""


## Visualization helper (provided)
It is not part of the protocol logic. Use it to see correlations and the CHSH value clearly.

In [ ]:
def visualize_bell_test_results(correlations, chsh_value, title="Bell Test Results"):
    fig = plt.figure(figsize=(12, 5))
    
    # Correlation heatmap
    ax1 = fig.add_subplot(121)
    alice_bases = sorted(set(b[0] for b in correlations.keys()))
    bob_bases = sorted(set(b[1] for b in correlations.keys()))
    corr_matrix = np.zeros((len(alice_bases), len(bob_bases)))
    
    for i, a_base in enumerate(alice_bases):
        for j, b_base in enumerate(bob_bases):
            if (a_base, b_base) in correlations:
                corr_matrix[i, j] = correlations[(a_base, b_base)]
    
    colors = [(0.8, 0.2, 0.2), (1, 1, 1), (0.2, 0.2, 0.8)]
    cmap = LinearSegmentedColormap.from_list('rwb', colors, N=100)
    
    im = ax1.imshow(corr_matrix, cmap=cmap, vmin=-1, vmax=1)
    ax1.set_title('Correlation Values E(a,b)')
    ax1.set_xticks(np.arange(len(bob_bases)))
    ax1.set_yticks(np.arange(len(alice_bases)))
    ax1.set_xticklabels([f"{b}°" for b in bob_bases])
    ax1.set_yticklabels([f"{a}°" for a in alice_bases])
    ax1.set_xlabel("Bob's Angle")
    ax1.set_ylabel("Alice's Angle")
    
    for i in range(len(alice_bases)):
        for j in range(len(bob_bases)):
            ax1.text(j, i, f"{corr_matrix[i, j]:.2f}", ha="center", va="center",
                    color="black" if abs(corr_matrix[i, j]) < 0.5 else "white")
    
    fig.colorbar(im, ax=ax1)
    
    # CHSH bar chart
    ax2 = fig.add_subplot(122)
    ax2.bar([0], [chsh_value], width=0.4, color='purple', alpha=0.7)
    ax2.axhline(y=2.0, color='r', linestyle='-', label='Classical Limit (2.0)')
    ax2.axhline(y=2*np.sqrt(2), color='b', linestyle='--', label='Quantum Limit (2√2)')
    ax2.set_ylim(0, 3.0)
    ax2.set_xticks([0])
    ax2.set_xticklabels(['CHSH Value'])
    ax2.text(0, chsh_value + 0.1, f"{chsh_value:.3f}", ha='center')
    ax2.legend(loc='upper left')
    ax2.set_title('CHSH Value vs Limits')
    
    verdict = " Entanglement!" if chsh_value > 2 else " No quantum"
    ax2.text(0, 1.0, verdict, ha='center', fontsize=14, fontweight='bold',
             color='green' if chsh_value > 2 else 'red')
    
    plt.suptitle(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

---
## 8) Generate Random Bases

Before we run the full test, we need a way to generate random choices for our measurement angles.


In [ ]:
def generate_random_bases(length: int, options: list) -> list:
    """
    Generate a list of random bases.
    """
    return [random.choice(options) for _ in range(length)]


---
## Run the Complete Bell Test!

Let's put it all together and verify quantum entanglement.

In [ ]:
# RUN THE COMPLETE BELL TEST

# Create 1000 Bell pairs (Alice and Bob share each pair)
NUM_PAIRS = 400
print(f"\nCreating {NUM_PAIRS} Bell pairs...")
bell_pairs = [create_bell_pair_singlet_state() for _ in range(NUM_PAIRS)]

# Run measurements on all pairs
print("Running measurements...")
results, alice_bases_used, bob_bases_used = run_bell_test_measurements(bell_pairs)

# Organize by basis pairs for correlation calculation
print("\nOrganizing results...")
organized = organize_measurements_by_basis(results, alice_bases_used, bob_bases_used)

# Calculate correlations
print("\nCalculating correlations...")
correlations = calculate_correlations(organized)

# Calculate CHSH
chsh = calculate_chsh_value(correlations)
print(f"\n" + "="*60)
print(f"CHSH VALUE: {chsh:.4f}")
print("="*60)

if check_bell_inequality(chsh):
    print("\n QUANTUM ENTANGLEMENT DETECTED!")
    print("   The Bell inequality is VIOLATED.")
    print("   This cannot be explained by classical physics!")
else:
    print("\n No quantum correlations detected.")
    print("   Check your implementation.")

In [ ]:
# Visualize
visualize_bell_test_results(correlations, chsh, "Bell Test: Singlet State |Ψ-⟩")

---
## Challenge: Detect Eavesdropping!

When Eve intercepts and measures the qubits, she destroys the entanglement.
So she must **send a replacement qubit** to Bob (otherwise he would notice missing photons).
In this simplified model, she **measures**, then **recreates a new product state** that matches her measurement result.

Let's see how this affects the CHSH value.


In [ ]:
def create_eavesdropped_state(bell_qc):
    """
    Simulate Eve measuring the Bell pair (destroys entanglement).
    """
    # Eve measures in Z basis
    eve_qc = bell_qc.copy()
    eve_qc.measure_all()
    eve_result = run_circuit(eve_qc, shots=1)
    eve_bits = list(eve_result.keys())[0]
    
    # Eve recreates a product state based on her measurement
    qc = QuantumCircuit(2)
    if eve_bits[0] == '1':  # qubit 1
        qc.x(1)
    if eve_bits[1] == '1':  # qubit 0
        qc.x(0)
    
    return qc

## Test with Eve intercepting 70% of pairs

In [ ]:
# Test with Eve intercepting 70% of pairs
EVE_PERCENTAGE = 0.7

print(f"\n EAVESDROPPING SIMULATION")
print(f"Eve intercepts {EVE_PERCENTAGE*100}% of Bell pairs\n")

# Create Bell pairs
print(f"Create Bell {NUM_PAIRS} pairs ...")
bell_pairs = [create_bell_pair_singlet_state() for _ in range(NUM_PAIRS)]

# Eve compromises some pairs
compromised = int(NUM_PAIRS * EVE_PERCENTAGE)
print(f"Number of compromised {compromised} / {NUM_PAIRS} ...")
eve_pairs = [create_eavesdropped_state(qc) for qc in bell_pairs[:compromised]] + bell_pairs[compromised:]

# Run Bell test on compromised pairs
print("Run circuit and get measurements ...")
results, alice_bases_used, bob_bases_used = run_bell_test_measurements(eve_pairs)

# Organize, calculate correlations and CHSH
organized = organize_measurements_by_basis(results, alice_bases_used, bob_bases_used)
correlations = calculate_correlations(organized)
chsh = calculate_chsh_value(correlations)

print(f"\nCHSH VALUE with Eve: {chsh:.4f}")

if check_bell_inequality(chsh):
    print("\n QUANTUM CORRELATIONS DETECTED!")
    print("   The system is entangled. The channel appears secure.")
    if EVE_PERCENTAGE > 0:
        print(f"   (Note: Eve was present at {EVE_PERCENTAGE*100}%, but didn't intercept enough to break the inequality.)")
else:
    print("\n EAVESDROPPING DETECTED!")
    print("   The Bell inequality is satisfied (S ≤ 2.0).")
    print("   Alice and Bob should abort: the connection is NOT secure.")


 EAVESDROPPING SIMULATION
Eve intercepts 70.0% of Bell pairs



NameError: name 'NUM_PAIRS' is not defined

In [ ]:

visualize_bell_test_results(correlations, chsh, f"Bell Test: {EVE_PERCENTAGE*100}% Eavesdropped")


<br><br><br>
# ==========================================
# PART 2: THE E91 PROTOCOL
# ==========================================


## E91 Protocol Key Points (Reminder)

- **Bases:** Alice $\{\color{red}{0^\circ}$, $45^\circ$, $\color{red}{90^\circ}\}$, Bob $\{\color{red}{45^\circ}$, $90^\circ$, $\color{red}{135^\circ}\}$
- **Key pairs:** $(45^\circ, 45^\circ)$ and $(90^\circ, 90^\circ)$
- **CHSH test pairs:** $(0^\circ, 90^\circ)$, $(0^\circ, 135^\circ)$, $(45^\circ, 90^\circ)$, $(45^\circ, 135^\circ)$
- **Discard:** all other basis combinations
- **Security:** compute $S$ from CHSH pairs; secure if $|S| > 2$
- **Singlet state:** outcomes are anti-correlated, so Bob flips bits for key

---

## Setup

**Important:** Make sure you completed and ran **all cells** in Notebook 1 first!

**Focus reminder:** most helper functions are familiar from CHSH; the **sifting step** is the heart of E91, so we will focus on that!


## 1) Generate Random Bases

Alice and Bob each randomly choose measurement bases for each qubit.
Why: independent, random choices prevent Eve from predicting measurement settings.


In [2]:
# Test
test_bases = generate_random_bases(10, ['0', '45', '90'])
print(f"Generated bases: {test_bases}")
print(f"Expected: 10 random values from ['0', '45', '90']")

NameError: name 'random' is not defined

## 2) Create Bell Pairs

Why: E91 starts with many identical entangled pairs (same idea as CHSH).


## 3) Measure All Pairs

## 4) Sift Results (Key Generation vs Bell Test)

**Sifting rules:**
- Same basis (45,45) or (90,90) → **Key generation**
- CHSH bases → **Bell test**
- Other combinations → **Discard**

**This is the heart of E91** — it decides which data becomes key vs security test.

Exercise: Implement `extract_e91_key_and_bell_test_data()`

### Sifting intuition (why it works)

- **Same basis** → strong (anti-)correlation → safe for key bits.
- **Different bases** → more random → useful for Bell test or discarded.

This mirrors what happens with polarizers in the lab experiment.

---

In [ ]:
def extract_e91_key_and_bell_test_data(results: list, alice_bases: list, bob_bases: list) -> dict:
    """
    Sift measurement results into key generation and Bell test data.
    
    Returns:
        dict with:
            'key_results': results for key generation
            'chsh_results': results for Bell test
            'chsh_alice_bases': Alice's bases for Bell test
            'chsh_bob_bases': Bob's bases for Bell test
    """
    key_results = [] # keep output results for same bases
    chsh_results = [] # keep output results for bases combinations that belong to CHSH test
    chsh_alice_bases = [] # keep alice bases that are in (combinations that belong to CHSH test)
    chsh_bob_bases = [] # same as alice
    # note : at index i, we have chsh output related to alice and bob base
    
    for i, (result, a_base, b_base) in enumerate(zip(results, alice_bases, bob_bases)):
        
        # TODO: Check if same basis (a_base == b_base)
        # If yes, add to key_results
        
        # TODO: Check if (a_base, b_base) is in CHSH_BASIS_PAIRS
        # If yes, add to chsh_results, chsh_alice_bases, chsh_bob_bases
        
        # Otherwise: discard
        pass
    
    return {
        'key_results': key_results,
        'chsh_results': chsh_results,
        'chsh_alice_bases': chsh_alice_bases,
        'chsh_bob_bases': chsh_bob_bases,
    }

## 5) Check for Eavesdropping

Use the Bell test data to verify security.
If $|S| > 2$, entanglement is intact and the channel is secure.

*(This reuses the CHSH logic from Notebook 1. Provided for speed.)*


In [ ]:
def check_for_eavesdropping(chsh_results: list, chsh_alice_bases: list, chsh_bob_bases: list) -> dict:
    """
    Run CHSH test to check for eavesdropping.
    """
    # 1. Organize the raw measurement results by basis pair
    bell_results = organize_measurements_by_basis(chsh_results, chsh_alice_bases, chsh_bob_bases)
    
    # 2. Calculate the correlation for each basis pair
    correlations = calculate_correlations(bell_results)
    
    # 3. Compute the CHSH 'S' value
    chsh_value = calculate_chsh_value(correlations)
    
    # 4. Check against the threshold (2.0)
    is_secure = chsh_value > BELL_INEQUALITY_THRESHOLD
    
    return {
        'chsh_value': chsh_value,
        'is_secure': is_secure
    }


## 6) Complete E91 Protocol

Put it all together!
Orchestration shows the full pipeline from entanglement to a shared key.

Exercise: Complete `run_e91_protocol()`

In [ ]:
EVE_PERCENTAGE_COMPROMISED = 0.7

def run_e91_protocol(num_pairs: int = 2000, eavesdropping: bool = False) -> str:
    """
    Run the complete E91 QKD protocol.
    
    Args:
        num_pairs: Number of Bell pairs to generate
        eavesdropping: If True, simulate Eve intercepting pairs
    
    Returns:
        Shared secret key (string of 0s and 1s) or None if insecure
    """
    print("="*60)
    print("E91 QUANTUM KEY DISTRIBUTION PROTOCOL")
    print("="*60)
    
    # Step 1: Generate Bell pairs
    print(f"\n1. Generating {num_pairs} Bell pairs...")
    bell_pairs = create_list_bell_pairs(num_pairs)
    
    # Step 2: Simulate eavesdropping (optional)
    if eavesdropping:
        compromised = int(num_pairs * EVE_PERCENTAGE_COMPROMISED)
        print(f"\n EVE intercepts {compromised} pairs ({EVE_PERCENTAGE_COMPROMISED*100}%)")
        bell_pairs = [create_eavesdropped_state(qc) for qc in bell_pairs[:compromised]] + bell_pairs[compromised:]
    
    # Step 3: Generate random bases
    print("\n2. Alice and Bob choose random bases...")
    alice_bases = generate_random_bases(num_pairs, ALICE_BASES)
    bob_bases = generate_random_bases(num_pairs, BOB_BASES)
    
    # Step 4: Measure all pairs
    print("\n3. Measuring all pairs...")
    results = measure_all_pairs(bell_pairs, alice_bases, bob_bases)
    
    # Step 5: Sift results
    print("\n4. Sifting results...")
    data = extract_e91_key_and_bell_test_data(results, alice_bases, bob_bases)
    
    print(f"   Key generation pairs: {len(data['key_results'])}")
    print(f"   Bell test pairs: {len(data['chsh_results'])}")
    
    # Step 6: Security check
    print("\n5. Running CHSH security test...")
    security = check_for_eavesdropping(
        data['chsh_results'],
        data['chsh_alice_bases'],
        data['chsh_bob_bases']
    )
    
    print(f"\n   CHSH Value: {security['chsh_value']:.4f}")
    print(f"   Classical limit: 2.0")
    
    if not security['is_secure']:
        print("\n SECURITY CHECK FAILED!")
        print("   Possible eavesdropping detected.")
        print("   Key generation ABORTED.")
        return None
    
    print("\n SECURITY CHECK PASSED!")
    
    # Step 7: Extract key
    print("\n6. Extracting shared key...")
    
    # Alice's bits (first bit of each result)
    alice_key = ''.join([r[0] for r in data['key_results']])
    
    # Bob's bits (second bit of each result)
    # For singlet state |Ψ-⟩: outcomes are anti-correlated, so Bob flips his bits.
    # HINT: If we were using a correlated state like |Φ-⟩, Bob would NOT flip his bits!
    bob_key = ''.join(['1' if r[1] == '0' else '0' for r in data['key_results']])
    
    # Verify keys match
    if alice_key != bob_key:
        mismatches = sum(a != b for a, b in zip(alice_key, bob_key))
        print(f"\n Warning: {mismatches} bit mismatches (noise or Eve)")
    else:
        print("\n Keys match perfectly!")
    
    print(f"\n   Key length: {len(alice_key)} bits")
    print(f"   Key (first 50 bits: {alice_key[:50]}...")
    
    return alice_key


##  Run E91 Protocol (No Eavesdropping)

In [ ]:
# Run without Eve
key = run_e91_protocol(num_pairs=2000, eavesdropping=False)

##  Run E91 Protocol (With Eavesdropping)

In [ ]:
# Reset seed
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)
MANUAL_SIMULATOR_SEED_COUNTER = GLOBAL_SEED

# Run with Eve
key_with_eve = run_e91_protocol(num_pairs=2000, eavesdropping=True)

## Encrypt and Decrypt Messages

In [ ]:
# use the generated key to encrypt and decrypt a message using XOR
# I use the key generated without Eve for encryption, as the one with Eve is likely compromised
message = '"What I cannot create, I do not understand." - Richard Feynman'
print(f"\nOriginal: {message}")

encrypted = enc.encrypt_xor_repeating_key(message, key)
print(f"\nEncrypted (hex): {encrypted[:50]}...")

decrypted = enc.decrypt_xor_repeating_key(encrypted, key)
print(f"\nDecrypted: {decrypted}")

---
## Challenge: Decrypt the Secret Messages!

The file `encrypted_messages.txt` contains messages encrypted with the E91 key.
Decrypt them!

In [ ]:
def decrypt_messages_from_file(key: str, filename: str):
    """Decrypt all messages from a file."""
    print(f"\nDecrypting messages from {filename}...\n")
    
    try:
        with open(filename, 'r') as f:
            for line in f:
                if ': ' in line:
                    msg_id, encrypted = line.strip().split(': ', 1)
                    decrypted = enc.decrypt_xor_repeating_key(encrypted, key)
                    print(f"{msg_id}: {decrypted}")
    except FileNotFoundError:
        print(f" File not found: {filename}")


# Decrypt the messages
if key:
    decrypt_messages_from_file(key, 'encrypted_messages.txt')

---
##  Summary

**What we implemented:**
1. Full E91 quantum key distribution protocol
2. Random basis selection for Alice and Bob
3. Result sifting (key vs Bell test)
4. CHSH security verification
5. Key extraction with anti-correlation handling
6. Message encryption/decryption

**Key insight:** The CHSH Bell test detects eavesdropping!
- No Eve → |S| ≈ 2.83 → Secure 
- With Eve → |S| drops → Detected! 

---

**Next:** Complete the take-home assignment using a **different Bell state**! 


<br><br><br>
# ==========================================
# PART 3: CODING CHALLENGE (ASSIGNMENT)
# ==========================================


# Coding Challenge - E91 with |Φ-⟩


In the workshop, we implemented E91 using the **singlet state** |Ψ-⟩ = (|01⟩ - |10⟩)/√2.

In this assignment, you will adapt E91 to use the **|Φ-⟩ Bell state** = (|00⟩ - |11⟩)/√2.

**Key difference:**
- |Ψ-⟩ gives **anti-correlated** results (Alice=0 → Bob=1)
- |Φ-⟩ gives **correlated** results (Alice=0 → Bob=0)


## What you will learn:

1. **Any Bell state works** for E91, but you need the right formula!
2. **Correlated vs Anti-correlated** states affect key extraction
3. **The CHSH formula changes** depending on the Bell state
4. **Experimental approach**: try all possibilities, find what gives 2√2

**TODO:** Adapt the E91 protocol to use the |Φ-⟩ Bell state and decrypt the secret messages.
- Create the |Φ-⟩ circuit (hint: add ONE gate to |Φ+⟩)
- Find the correct CHSH formula (try all 4 minus positions!)
- Complete E91 and decrypt the messages


### Setup

## Task 1: Create |Φ-⟩ Bell State

Complete the function to create |Φ-⟩ = (|00⟩ - |11⟩)/√2

**Hint:** Start with |Φ+⟩ and add ONE gate to create the minus sign!

```
|Φ+⟩: H → CX       gives (|00⟩ + |11⟩)/√2
|Φ-⟩: H → ? → CX   gives (|00⟩ - |11⟩)/√2
```

**Remember:** The Z gate adds a π phase: |1⟩ → -|1⟩


In [ ]:
def create_bell_pair_phi_minus() -> QuantumCircuit:
    """
    TODO: Create the Bell state |Φ-⟩ = (|00⟩ - |11⟩)/√2
    
    Hint: Start with |Φ+⟩ (H + CX) and add ONE gate to create the minus sign!
    """
    qc = QuantumCircuit(2)
    
    # TODO: Add your gates here
    # qc.h(0)     # Hadamard on qubit 0
    # qc.?        # What gate creates the minus sign?
    # qc.cx(0,1)  # CNOT
    
    return qc

In [ ]:
# Test your circuit
phi_minus = create_bell_pair_phi_minus()
print("Your circuit for |Φ-⟩:")
print(phi_minus.draw())

### Test the |Φ-⟩ State

Make sure you created the correct Bell state.  
Tip: $1/\sqrt{2}$ is the same as $\sqrt{2}/2$ 😄


In [ ]:
from qiskit.quantum_info import Statevector
Statevector(phi_minus).draw('latex')


## Verify: Test Your |Φ-⟩ Circuit

Run this to check if your circuit is correct. You should see ~50% |00⟩ and ~50% |11⟩.


In [ ]:
# Test the |Φ-⟩ Bell state
phi_minus = create_bell_pair_phi_minus()

# Verify with measurements
test_qc = phi_minus.copy()
test_qc.measure_all()
counts = run_circuit(test_qc, shots=1000)

print(f"\nMeasurement results: {counts}")
print("\n Expected: ~50% |00⟩ and ~50% |11⟩ (never |01⟩ or |10⟩)")
print("   This confirms |Φ-⟩ is CORRELATED : ")
print("   --> Alice and Bob get SAME results: Alice=0 → Bob=0, Alice=1 → Bob=1")

print("\n Note: The minus sign (phase) doesn't affect Z-basis measurements,")
print("   but it DOES affect measurements in rotated bases (used for CHSH)!")


## Task 2: Find the Correct CHSH Formula

The CHSH formula has ONE minus sign. For |Φ-⟩, it moves to a different position!

**Workshop formula (for |Ψ-⟩):**
$$S = E(a,b) - E(a,b') + E(a',b) + E(a',b')$$

**Your task:** Try all 4 positions for the minus sign and find which gives |S| ≈ 2√2 ≈ 2.83


**Fisrt we generate test data for |Φ-⟩ :**

In [ ]:
# Generate test data for |Φ-⟩

print("Generating 500 measurements for CHSH test...\n")
test_num_pairs = 500

# Generate random bases
test_alice_chsh_bases = [random.choice(ALICE_CHSH_BASES) for _ in range(test_num_pairs)]
test_bob_chsh_bases = [random.choice(BOB_CHSH_BASES) for _ in range(test_num_pairs)]

# Generate Bell pairs and Measure them
results = []
for a, b in zip(test_alice_chsh_bases, test_bob_chsh_bases):
    qc = create_bell_pair_phi_minus()
    results.append(measure_bell_pair(qc, a, b))


# step 4: Organize measurements by basis to compute correkations:
measurements = organize_measurements_by_basis(results, test_alice_chsh_bases, test_bob_chsh_bases)
correlations = calculate_correlations(measurements)

# Print correlations
print("Individual correlations E(a,b):")
for pair, E in sorted(correlations.items()):
    print(f"  E{pair} = {E:.4f}")

# Step 5: TODO
# Based on the correlations, compute and find the correct CHSH formulat (in next cell)

In [ ]:
# ═══════════════════════════════════════════════════════════
# TODO: Try all 4 minus sign positions!
# ═══════════════════════════════════════════════════════════

a1, a2 = '0', '90'   # Alice's CHSH bases
b1, b2 = '45', '135' # Bob's CHSH bases

E_ab = correlations[(a1, b1)]              # E(0°, 45°)
E_ab_prime = correlations[(a1, b2)]        # E(0°, 135°)
E_a_prime_b = correlations[(a2, b1)]       # E(90°, 45°)
E_a_prime_b_prime = correlations[(a2, b2)] # E(90°, 135°)

print("="*60)
print("Testing all 4 CHSH formula variations:")
print("="*60)

# TODO: Calculate S for each position of the minus sign

# Position 1: minus at first term
S_minus_p1 = 0  # TODO: -E_ab + E_ab_prime + E_a_prime_b + E_a_prime_b_prime
print(f"\n1. S = -E(a,b) + E(a,b') + E(a',b) + E(a',b')")
print(f"   |S| = {abs(S_minus_p1):.4f}")

# Position 2: minus at second term (original workshop formula)
S_minus_p2 = 0  # TODO: E_ab - E_ab_prime + E_a_prime_b + E_a_prime_b_prime
print(f"\n2. S = +E(a,b) - E(a,b') + E(a',b) + E(a',b')  [workshop formula]")
print(f"   |S| = {abs(S_minus_p2):.4f}")

# Position 3: minus at third term
S_minus_p3 = 0  # TODO: E_ab + E_ab_prime - E_a_prime_b + E_a_prime_b_prime
print(f"\n3. S = +E(a,b) + E(a,b') - E(a',b) + E(a',b')")
print(f"   |S| = {abs(S_minus_p3):.4f}")

# Position 4: minus at fourth term
S_minus_p4 = 0  # TODO: E_ab + E_ab_prime + E_a_prime_b - E_a_prime_b_prime
print(f"\n4. S = +E(a,b) + E(a,b') + E(a',b) - E(a',b')")
print(f"   |S| = {abs(S_minus_p4):.4f}")

print("\n" + "="*60)
print(f"Target: 2√2 ≈ {2*np.sqrt(2):.4f}")
print("The formula with |S| ≈ 2.83 is the CORRECT one!")
print("="*60)

### Your Answer: Which CHSH formula is correct?

Based on your results above, write down the correct CHSH formula for |Φ-⟩:


In [ ]:
def calculate_chsh_value_phi_minus(correlations, alice_bases=ALICE_CHSH_BASES, bob_bases=BOB_CHSH_BASES):
    """
    TODO: Implement the correct CHSH formula for |Φ-⟩
    
    Based on your experiment above, which position gives |S| ≈ 2√2?
    """
    a1, a2 = alice_bases  # '0', '90'
    b1, b2 = bob_bases    # '45', '135'
    
    # TODO: Write the correct formula here
    # S = ???
    S = 0  # Replace with the correct formula!
    
    
    return abs(S)


# Test your formula
chsh = calculate_chsh_value_phi_minus(correlations)
print(f"Your CHSH value: {chsh:.4f}")
print(f"Expected: ≈ {2*np.sqrt(2):.4f}")

---
## E91 Protocol Helper Functions

These functions orchestrate the full E91 key distribution protocol.



## Task 3: Complete E91 Protocol and Decrypt Messages

Now put it all together! Complete the `run_e91_protocol` function.

**Remember:**
- Use your |Φ-⟩ Bell state
- Use your correct CHSH formula
- |Φ-⟩ is **correlated** → Bob uses bits directly (no flip!)


In [ ]:
# ============================================================
# COMPLETE E91 PROTOCOL with |Φ-⟩
# ============================================================

# Reset seed for reproducibility
MANUAL_SIMULATOR_SEED_COUNTER = 42
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

def run_e91_protocol(num_pairs=2000):
    """
    Run E91 protocol with |Φ-⟩ Bell state.
    
    TODO: Complete this function using:
    - Your create_bell_pair_phi_minus() function
    - Your calculate_chsh_value_phi_minus() function
    - Correct key extraction for CORRELATED state
    """
    print("="*60)
    print("E91 PROTOCOL with |Φ-⟩ Bell State")
    print("="*60)
    
    # Step 1: Generate Bell pairs
    print(f"\n1. Generating {num_pairs} |Φ-⟩ Bell pairs...")
    bell_pairs = create_list_bell_pairs(num_pairs)
    
    # Step 2: Generate random bases
    print("\n2. Alice and Bob choose random bases...")
    alice_bases = generate_random_bases(num_pairs, ALICE_BASES)
    bob_bases = generate_random_bases(num_pairs, BOB_BASES)
    
    # Step 3: Measure all pairs
    print("\n3. Measuring all pairs...")
    results = measure_all_pairs(bell_pairs, alice_bases, bob_bases)
    
    # Step 4: Sift results
    print("\n4. Sifting results...")
    data = extract_e91_key_and_bell_test_data(results, alice_bases, bob_bases)
    print(f"   Key generation pairs: {len(data['key_results'])}")
    print(f"   CHSH test pairs: {len(data['chsh_results'])}")
    
    # Step 5: Security check
    print("\n5. Running CHSH security test...")
    security = check_for_eavesdropping(
        data['chsh_results'],
        data['chsh_alice_bases'],
        data['chsh_bob_bases']
    )
    print(f"\n   CHSH Value: {security['chsh_value']:.4f}")
    print(f"   Classical limit: 2.0")
    print(f"   Quantum limit: 2√2 ≈ 2.83")
    
    if not security['is_secure']:
        print("\n SECURITY CHECK FAILED!")
        print("   Possible eavesdropping detected.")
        return None
    
    print("\n SECURITY CHECK PASSED!")
    
    # Step 6: Extract key
    print("\n6. Extracting shared key...")
    
    # TODO: Key extraction for |Φ-⟩
    # Remember: |Φ-⟩ is CORRELATED → Bob uses bits DIRECTLY (no flip!)
    # Qiskit format: result = "BA" where B=qubit1, A=qubit0
    
    alice_key = ''.join([r[1] for r in data['key_results']])  # Alice = qubit 0
    bob_key = None    ## r[0] = Bob (qubit 1) - TODO: flip or not?

    
    if alice_key == bob_key:
        print("\n Keys match perfectly!")
    else:
        mismatches = sum(a != b for a, b in zip(alice_key, bob_key))
        print(f"\n {mismatches} mismatches in {len(alice_key)} bits")
    
    print(f"   Key length: {len(alice_key)} bits")
    print(f"   Key (first 50): {alice_key[:50]}...")
    
    return alice_key


print(" E91 Protocol function loaded!")

### Run E91 Protocol


In [ ]:
key = run_e91_protocol(num_pairs=2000)


### Decrypt the Secret Messages

If your protocol worked correctly, you should be able to decrypt the messages!


In [ ]:
def decrypt_and_print_messages(key: str, filename: str = "encrypted_messages.txt"):
    """
    Read encrypted messages from a file, decrypt them using XOR with the given key, and print.

    Args:
        key (str): The key to use for XOR decryption.
        filename (str): The file to read encrypted messages from.
    """
    print("\nDecrypting all messages from", filename)
    with open(filename, "r") as f:
        for line in f:
            if ": " in line:
                msg_id, encrypted = line.split(": ", 1)
                decrypted = enc.decrypt_xor_repeating_key(encrypted.strip(), key)
                print(f"{msg_id}: {decrypted}")


In [ ]:
# For your challenge: must generate the correct key using the E91 protocol to decrypt the messages. Without the key, decryption is not feasible.
# change the path to the encrypted file as needed
path_to_encrypted_file = R"assignment_encrypted_messages.txt"
decrypt_and_print_messages(key, filename=path_to_encrypted_file)



## Summary: What You Learned

Complete this table based on your work:

| Aspect             | Workshop ($\vert \Psi^- \rangle$)         | This Assignment ($\vert \Phi^- \rangle$) |
|-------------------|-------------------------------------------|-----------------------------------------|
| Circuit            | H → CX → X → Z                             | H → ??? → CX                            |
| Correlation        | Anti-correlated                             | ???                                     |
| Key extraction     | Bob flips bits                              | ???                                     |
| CHSH formula       | $S = +E(a,b) - E(a,b') + E(a',b) + E(a',b')$ | $S = ???$                              |

**Key insight:** Any Bell state can be used for E91, but you need to:  
1. Know if it's **correlated** or **anti-correlated** (for key extraction)  
2. Find the **correct CHSH formula** (the one that gives $|S| \approx 2\sqrt{2}$)
